# What is Chatbot?

A chatbot is an intelligent piece of software that is capable of communicating and performing actions similar to a human. Chatbots are used a lot in customer interaction, marketing on social network sites and instantly messaging the client. There are two basic types of chatbot models based on how they are built; Retrieval based and Generative based models.

## 1. Retrieval based Chatbots
A retrieval-based chatbot uses predefined input patterns and responses. It then uses some type of heuristic approach to select the appropriate response. It is widely used in the industry to make goal-oriented chatbots where we can customize the tone and flow of the chatbot to drive our customers with the best experience.

## 2. Generative based Chatbots
Generative models are not based on some predefined responses.

They are based on seq 2 seq neural networks. It is the same idea as machine translation. In machine translation, we translate the source code from one language to another language but here, we are going to transform input into an output. It needs a large amount of data and it is based on Deep Neural networks.

## About the Python Project – Chatbot
In this Python project with source code, I am going to build a chatbot using deep learning techniques. The chatbot will be trained on the dataset which contains categories (intents), pattern and responses. We use a special recurrent neural network (LSTM) to classify which category the user’s message belongs to and then we will give a random response from the list of responses.

Let’s create a retrieval based chatbot using NLTK, Keras, Python, etc.

The dataset we will be using ‘intents.json’. This is a JSON file that contains the patterns we need to find and the responses we want to return to the user.


In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
#pip install tensorflow --user

In [9]:
#pip install keras --user

In [10]:
#pip install pickle --user

In [11]:
#pip install nltk --user

## How to Make Chatbot in Python
Now we are going to build the chatbot using Python but first, let us see the file structure and the type of files

- Intents.json – The data file which has predefined patterns and responses.<br/>
- train_chatbot.py – In this Python file, we wrote a script to build the model and train our chatbot.<br/>
- Words.pkl – This is a pickle file in which we store the words Python object that contains a list of our vocabulary.<br/>
- Classes.pkl – The classes pickle file contains the list of categories.<br/>
- Chatbot_model.h5 – This is the trained model that contains information about the model and has weights of the neurons.<br/>
- Chatgui.py – This is the Python script in which we implemented GUI for our chatbot. Users can easily interact with the bot.<br/>

**Here are the 5 steps to create a chatbot in Python from scratch:**<br/>

- Import and load the data file<br/>
- Preprocess data<br/>
- Create training and testing data<br/>
- Build the model<br/>
- Predict the response<br/>

**1. Import and load the data file**

We import the necessary packages for our chatbot and initialize the variables we will use in our project.

The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents_tj_ru.json', encoding="utf8").read()
intents = json.loads(data_file)

This is how our intents.json file looks like.

In [13]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'привет',
   'patterns': ['Привет',
    'Здравствуй',
    'Здравствуйте',
    'Эй',
    'Есть кто нибудь?',
    'Алло',
    'Тук-тук'],
   'responses': ['Привет, спасибо что спрашиваете',
    'Рад вас видеть',
    'Привет, чем могу помочь?'],
   'context': ['']},
  {'tag': 'салом',
   'patterns': ['Салом',
    'Саломалейкум',
    'Чи хел?',
    'Ҳей',
    'Ягон кас хаст?',
    'Алло',
    'Ассалому алейкум'],
   'responses': ['Салом, ба шумо чи хизмат кунем?',
    'Мо аз дидани шумо хушҳол шудем',
    'Салом, чи тавр ёри расонем?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'resp

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kurbonovf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**2. Preprocess data**

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the data.

Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [15]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kurbonovf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [17]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

3572 documents
14 classes ['goodbye', 'greeting', 'options', 'product_search', 'search_product_by_name', 'thanks', 'варианты', 'вариантҳо', 'пока', 'привет', 'раҳмат', 'салом', 'спасибо', 'хайр']
2261 unique lemmatized words ["'s", '(', ')', ',', '.', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'find', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'in', 'is', 'later', 'list', 'me', 'medicament', 'next', 'nice', 'of', 'offered', 'pharmacy', 'pill', 'provide', 'search', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'you', 'абактал', 'авамис', 'авексон-рационал', 'аводарт', 'авокадо', 'аврора', 'автопассит', 'авторекс', 'авторекс-а', 'агри', 'адамол', 'адаптол', 'адвантан', 'аджисек', 'аджисепт', 'адикардин', 'адонис-бром', 'аевит', 'азатиоприн', 'азафен', 'азимак', 'азитромицин', 'азопт', 'аир', 'айдип', 'айтон', 'аква', 'аквадетрим', 'аквалор', 'акизол-40', 'ак

**3. Create training and testing data**

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [18]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


**4. Build the model**

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [19]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
715/715 [==============================] - 3s 3ms/step - loss: 0.2123 - accuracy: 0.9773
Epoch 2/200
715/715 [==============================] - 2s 3ms/step - loss: 0.1506 - accuracy: 0.9798
Epoch 3/200
715/715 [==============================] - 2s 3ms/step - loss: 0.1293 - accuracy: 0.9798
Epoch 4/200
715/715 [==============================] - 2s 3ms/step - loss: 0.1148 - accuracy: 0.9801
Epoch 5/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0960 - accuracy: 0.9804
Epoch 6/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0855 - accuracy: 0.9801
Epoch 7/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0663 - accuracy: 0.9815
Epoch 8/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0615 - accuracy: 0.9826
Epoch 9/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0557 - accuracy: 0.9815
Epoch 10/200
715/715 [==============================] - 2s 3ms/step - loss: 0.0508 - accura

**5. Predict the response (Graphical User Interface)**


To predict the sentences and get a response from the user to let us code the following.

We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents_tj_ru.json', encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [21]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

We read the medicaments from Excel file. The excel file was extracted using pharmacy software which is developed in C# by our company.

In [22]:
from pathlib import Path
import openpyxl

data = {}

def read_xls():
    xlsx_file = Path('C:\Other\HSE\Course\ML Project\chatbot-python-project-data-codes', 'Product.xlsx')
    wb_obj = openpyxl.load_workbook(xlsx_file) 

    # Read the active sheet:
    sheet = wb_obj.active

    for i, row in enumerate(sheet.iter_rows(values_only=True)):
        if i == 0:
            data['Product'] = []
            data['Quantity'] = []
            data['Price'] = []
            data['DueTo'] = []
            data['Manufacturer'] = []
        else:
            data['Product'].append(row[0])
            data['Quantity'].append(row[2])
            data['Price'].append(row[3])
            data['DueTo'].append(row[4])
            data['Manufacturer'].append(row[5])

    return data

After predicting the class, we will get a random response from the list of intents.

In [37]:
def getResponse(ints, intents_json, text):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']=="search_product_by_name"):
            result = search_by_name(text)
            break        
        elif(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents, text)
    return res

def search_by_name(text):
    read_xls()
    for i in range(len(data['Product'])):
        if data['Product'][i] == text:
            return "Наименование: "+data['Product'][i] + "  Количество: "+str(data['Quantity'][i]) + "  Цена: "+str(data['Price'][i]) + " сомони" + "  Срок годности: "+data['DueTo'][i].strftime('%d-%m-%Y') + "  Производитель: "+str(data['Manufacturer'][i])
        elif i==len(data['Product'])-1:
            return "Не нашёл запрашиваемый препарат, попробуйте другое наименование препарата"

Now we will develop a graphical user interface. Let’s use Tkinter library which is shipped with tons of useful libraries for GUI. We will take the input message from the user and then use the helper functions we have created to get the response from the bot and display it on the GUI. Here is the full source code for the GUI.

In [38]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

The program will open up a GUI window within a few seconds. With the GUI you can easily chat with the bot.

## Summary
In this project, we understood about chatbots and implemented a deep learning version of a chatbot in Python which is accurate. You can customize the data according to business requirements and train the chatbot with great accuracy. Chatbots are used everywhere and all businesses are looking forward to implementing bot in their workflow.
